# Unit Tests for Pipeline
This notebook contains the unit tests for the Pipeline

## Import dependencies for JUnit Output
We'll install the PyPI library and restart Python so we can use it. Alternatively, install this in the cluster.

In [2]:
dbutils.library.installPyPI('unittest-xml-reporting')
dbutils.library.restartPython()

## Imports

In [4]:
from pyspark.sql import Row

import unittest
import xmlrunner
import uuid
import io

## Call the Pipeline notebook

The next cell will execute the Pipeline notebook. This will create all of the classes and other definitions in the scope of this notebook so we can reference them later.

In [6]:
%run ./pipeline

## Create some mock classes
To promote well structured and testable code, we have mocked the DBUtils and a FileAccess class so we can abstract the implementation of environment specific details from the system under test. In this case, we're interested in testing the **Pipeline** class and not whether Databricks can read data from storage.

In [8]:
class MockDbUtilsSecrets:
  def __init__(self, secrets: dict) -> None:
    self.secrets = secrets
    
  def get(self, scope: str, key: str) -> str:
    return self.secrets.get(scope, {}).get(key, '')

class MockDbUtils:
  def __init__(self, secrets: dict) -> None:
    self.secrets = MockDbUtilsSecrets(secrets)
    
class MockFileAccess(FileAccess):
  def read(self, path: str) -> DataFrame:
    self.source = self.spark.createDataFrame([Row(id=1, name='Item 1', value=str(uuid.uuid4()))])
    return self.source.cache()
    
  def write(self, path: str, df: DataFrame) -> None:
    self.target = df
    

## Create our tests class
The following class contains our unit tests we wish to execute to prove the Pipeline class functions as expeceted.

In [10]:
class PipelineTests(unittest.TestCase):
  @classmethod
  def setUpClass(cls):
    dbutils = MockDbUtils({
    })
    files = MockFileAccess(spark, dbutils)
    pipeline = Pipeline(spark, files)
    pipeline.run()    
    cls.source = files.source.collect()
    cls.records = files.target.collect()
    
  def test_record_count(self):
    self.assertEqual(1, len(self.records), 'There should be a single row')
    
  def test_name_transformation(self):
    self.assertEqual('ITEM 1', self.records[0].name, 'The name should be uppercase')
    
  def test_input_and_output_value_matches(self):
    self.assertEqual(self.source[0].value, self.records[0].value, 'The value should match')
    
  def test_record_is_processed(self):
    self.assertEqual(1, self.records[0].processed, 'The record should be processed')


## Execute the Tests
The following cell creates a TestSuite and then executes that to create JUnit XML output. This output is written to the DBFS file system to facilitate CI/CD pipelines.

The process to include in CI/CD is:
* deploy the pipeline, driver and tests notebooks in a folder
* execute the tests notebook
* collect the JUnit XML output
* publish the test results

In [12]:
loader = unittest.TestLoader()
suite = unittest.TestSuite()
tests = loader.loadTestsFromTestCase(PipelineTests)
suite.addTests(tests)

out = io.BytesIO()
runner = xmlrunner.XMLTestRunner(out)
runner.run(suite)
out.seek(0)

with open('/dbfs/tmp/pipeline/junit/TEST-Pipeline.xml', 'wb') as f:
  f.write(out.read())

Running tests...
----------------------------------------------------------------------
....
----------------------------------------------------------------------
Ran 4 tests in 6.949s

OK

Generating XML reports...

In [13]:
out.seek(0)
print(out.read().decode('utf-8'))

<?xml version="1.0" encoding="UTF-8"?>
<testsuites>
	<testsuite errors="0" failures="0" file=".py" name="PipelineTests-20200713163448" skipped="0" tests="4" time="0.003" timestamp="2020-07-13T16:34:55">
 <testcase classname="PipelineTests" file="/local_disk0/tmp/1594658076474-0/PythonShell.py" line="18" name="test_input_and_output_value_matches" time="0.002" timestamp="2020-07-13T16:34:55">
 <system-out>
<![CDATA[]]> </system-out>
 <system-err>
<![CDATA[]]> </system-err>
 </testcase>
 <testcase classname="PipelineTests" file="/local_disk0/tmp/1594658076474-0/PythonShell.py" line="15" name="test_name_transformation" time="0.001" timestamp="2020-07-13T16:34:55">
 <system-out>
<![CDATA[]]> </system-out>
 <system-err>
<![CDATA[]]> </system-err>
 </testcase>
 <testcase classname="PipelineTests" file="/local_disk0/tmp/1594658076474-0/PythonShell.py" line="12" name="test_record_count" time="0.001" timestamp="2020-07-13T16:34:55">
 <system-out>
<![CDATA[]]> </system-out>
 <system-err>
<![CDATA[]]> </system-err>
 </testcase>
 <testcase classname="PipelineTests" file="/local_disk0/tmp/1594658076474-0/PythonShell.py" line="21" name="test_record_is_processed" time="0.001" timestamp="2020-07-13T16:34:55">
 <system-out>
<![CDATA[]]> </system-out>
 <system-err>
<![CDATA[]]> </system-err>
 </testcase>
	</testsuite>
</testsuites>

In [14]:
PipelineTests.__dict__

Out[10]: mappingproxy({'__module__': '__main__',
 'setUpClass': <classmethod at 0x7fb095e4d898>,
 'test_record_count': <function __main__.PipelineTests.test_record_count(self)>,
 'test_name_transformation': <function __main__.PipelineTests.test_name_transformation(self)>,
 'test_input_and_output_value_matches': <function __main__.PipelineTests.test_input_and_output_value_matches(self)>,
 'test_record_is_processed': <function __main__.PipelineTests.test_record_is_processed(self)>,
 '__doc__': None,
 '_classSetupFailed': False,
 'source': [Row(id=1, name='Item 1', value='d80550bc-6aeb-4a50-9f3a-1f0905bbabc3')],
 'records': [Row(id=1, name='ITEM 1', value='d80550bc-6aeb-4a50-9f3a-1f0905bbabc3', processed=1, processed_time=datetime.datetime(2020, 7, 13, 16, 34, 55, 242000))]})